In [1]:
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np
from sys import stdout
from openmmtools import integrators
import mdtraj
from openmmtools import testsystems


In [2]:
pdb = app.PDBFile('ala2_fromURL.pdb')
topology = pdb.getTopology()
positions = pdb.getPositions()


ff = app.ForceField('amber99sbildn.xml',"amber96_obc.xml")
system = ff.createSystem(
    topology=topology, 
    constraints=app.HBonds, 
    rigidWater=True
    )

platform = openmm.Platform.getPlatform(2)



t = testsystems.AlanineDipeptideImplicit()
system, positions, topology = t.system, t.positions, t.topology
platform = openmm.Platform.getPlatform(2)

for i in range(0,system.getNumConstraints()):
    print(system.getConstraintParameters(i))
    constrained_idx = (system.getConstraintParameters(i)[0:2])
    for atom in topology.atoms():
        if atom.index in constrained_idx:
            print(f'Atom name: {atom.name} /n Atom type: {atom.element}')    


In [3]:
temperature = 300.0 * unit.kelvin
collision_rate = 1.0 / unit.picosecond
timestep = 4.0 * unit.femtosecond
reportInterval = 2500
steps = 2.5E+11
#time = (steps*timestep).value_in_unit(unit.nanosecond)
parameters = f'{temperature}\nCollision rate:{collision_rate}\nTimestep: {timestep}\nReport every {reportInterval} steps'
with open('parameterfile.txt', 'w') as f:
    f.write(parameters)

In [5]:

from mdtraj import reporters

integrator = integrators.LangevinIntegrator(temperature=temperature,collision_rate=collision_rate,timestep=timestep)
#integrator.setConstraintTolerance(0.00001)
#integrator = openmm.VerletIntegrator(timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "2"
simulation = app.Simulation(topology, system, integrator,platform,platformProperties=properties_dict)
#print(platform.getPropertyValue(simulation.context,property='Precision'))
#print(simulation.context.getPlatform())
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)

simulation.reporters.append(app.StateDataReporter(stdout, 1000000, step=True, potentialEnergy=True,temperature=True))
simulation.reporters.append(app.DCDReporter('obcimplicit.dcd',reportInterval))
#h5_reporter = reporters.HDF5Reporter('output.h5',reportInterval)
#simulation.reporters.append(h5_reporter)


In [6]:
simulation.step(steps)
#h5_reporter.close()

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000000,-76.19180297851562,188.13529052629633
2000000,-68.78399658203125,248.6319069874564
3000000,-78.21420288085938,308.95837064805437
4000000,-74.51077270507812,229.29519360781134
5000000,-66.74346923828125,176.22241742351818
6000000,-79.58282470703125,277.69494811787473
7000000,-81.13861083984375,241.5101747442406
8000000,-80.20535278320312,231.8457043825265
9000000,-66.44058227539062,256.0938368782378
10000000,-86.59893798828125,253.9493754218145
11000000,-91.82418823242188,222.46221914150377
12000000,-53.236663818359375,320.2342997630247
13000000,-91.77764892578125,292.1785552953397
14000000,-61.87969970703125,244.73253449070043
15000000,-70.994384765625,167.42089182447293
16000000,-64.07241821289062,314.8656973264251
17000000,-80.40725708007812,240.4197178568908
18000000,-77.14251708984375,213.52159538224436
19000000,-88.32772827148438,270.0794183055115
20000000,-77.76373291015625,253.52232913638133
21000000,-56.571990966796

traj = mdtraj.load('output.pdb')
import nglview
view = nglview.show_mdtraj(traj)
view